https://docs.datacentral.org.au/galah/dr3/overview/

In [ ]:
import webbrowser

webbrowser.open('https://docs.datacentral.org.au/galah/dr3/overview/')

webbrowser.open('https://cloud.datacentral.org.au/teamdata/GALAH/public/GALAH_DR3/old_version_1/')

webbrowser.open('https://cloud.datacentral.org.au/teamdata/GALAH/public/GALAH_DR3/')

#webbrowser.open('https://docs.datacentral.org.au/help-center/virtual-observatory-examples/ssa-galah-dr3-interactive-spectra-explorer-enhanced-data-central-api/')

In [ ]:
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [1]:
import sys
print(sys.executable)

import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

import matplotlib.pyplot as plt
from matplotlib import gridspec

import numpy as np
import pandas as pd

from specutils import Spectrum1D
from pyvo.dal.ssa  import search, SSAService

import requests
from io import BytesIO

import matplotlib.ticker as plticker

from astropy.stats import sigma_clip


import starcolorindexSpT

/Users/josephkarpinski/opt/anaconda3/envs/envGalah/bin/python


In [2]:
pd.set_option('display.max_rows', 2000)

In [3]:
Ipython_default = plt.rcParams.copy()

#plt.style.use('dark_background')

# reset rcParams
#plt.rcParams.update(Ipython_default)

In [4]:
#set the figure size to be wide in the horizontal direction
#to accommodate the image cutouts alongside the spectra 
#fsize=[20,13]
fsize=[20,8]
#mpl.rcParams['axes.linewidth'] = 0.7
FSIZE=18
LWIDTH=0.5
LABELSIZE=10

In [5]:
def show_spectrum(name,axes,title): 
    #query the SSA service
    url = "https://datacentral.org.au/vo/ssa/query"
    service = SSAService(url)
    custom = {}
    custom['TARGETNAME'] = name
    custom['COLLECTION'] = 'galah_dr3'
    results = service.search(**custom)
    df = results.votable.get_first_table().to_table(use_names_over_ids=True).to_pandas()
    filters = ['B','V','R','I']
    colours = ["#085dea","#1e8c09","#cf0000","#640418"]

    #go through each filter and plot its spectrum (if available)
    for idx in range(0,4):
        filt = filters[idx]
        ax = axes[idx]
        ax.clear()
        #show the title in the first position only
        if(idx == 0):
            ax.set_title(title)
        #select only the spectrum of the filter of interest
        subset = df[(df['band_name'] == filt)].reset_index()
        #give preference to using the continuum normalised spectra 
        if(subset[subset['dataproduct_subtype'].isin(['normalised'])].shape[0] > 0):
            subset = subset[(subset['dataproduct_subtype'] == "normalised")].reset_index()
        #only proceed if we have the filter of interest available in the results 
        if(subset.shape[0] > 0):
            #add RESPONSEFORMAT=fits here to ensure we get fits format back
            url= subset.loc[0,'access_url'] + "&RESPONSEFORMAT=fits"
            #load the spectrum
            spec = Spectrum1D.read(BytesIO(requests.get(url).content),format='wcs1d-fits')
            #print(spec)
            #df_spec = pd.DataFrame(spec)
            #df_spec.to_csv("df_spec_" + str(idx))
            exptime = subset.loc[0,'t_exptime']
            ax.tick_params(axis='both', which='major')
            ax.tick_params(axis='both', which='minor')
            loc = plticker.MultipleLocator(base=20.0)
            ax.xaxis.set_major_locator(loc)
            #plot label at last position (caveat: no check if spectra are missing)
            if(idx == 3): 
                ax.set_xlabel("Wavelength ($\mathrm{\AA}$)",labelpad=10)
            #plot the spectrum 
            ax.plot(spec.wavelength,spec.flux,linewidth=LWIDTH,color=colours[idx])
            #adjust the y-scale to best fit the spectrum with some sigma clipping
            nspec = len(spec.spectral_axis)
            clipped = sigma_clip(spec[int(0+0.01*nspec):int(nspec-0.01*nspec)].flux,masked=False,sigma=15)
            ymin = min(clipped).value
            ymax = max(clipped).value 
            xmin = spec.wavelength[0].value
            xmax = spec.wavelength[nspec-1].value
            #add a 1% buffer either side of the x-range
            dx=0.01*(xmax-xmin)
            ax.set_xlim(xmin-dx,xmax+dx)
            #add a 1% buffer either side of the y-range
            dy=0.03*(ymax-ymin)
            ax.set_ylim(ymin-dy,ymax+dy)
        #else:
            #print missing data...

In [6]:
# GALAH DR3 Version 1 
# https://cloud.datacentral.org.au/teamdata/GALAH/public/GALAH_DR3/old_version_1/

df_GALAH = pd.read_csv('GALAH_DR3_main_allspec_v1.csv')

In [28]:
df_GALAH.columns.tolist()

['star_id',
 'sobject_id',
 'source_id',
 'survey_name',
 'field_id',
 'flag_repeat',
 'wg4_field',
 'wg4_pipeline',
 'flag_sp',
 'teff',
 'e_teff',
 'irfm_teff',
 'irfm_ebv',
 'irfm_ebv_ref',
 'logg',
 'e_logg',
 'fe_h',
 'e_fe_h',
 'flag_fe_h',
 'fe_h_atmo',
 'vmic',
 'vbroad',
 'e_vbroad',
 'chi2_sp',
 'alpha_fe',
 'e_alpha_fe',
 'nr_alpha_fe',
 'flag_alpha_fe',
 'flux_A_Fe',
 'chi_A_Fe',
 'Li_fe',
 'e_Li_fe',
 'nr_Li_fe',
 'flag_Li_fe',
 'C_fe',
 'e_C_fe',
 'nr_C_fe',
 'flag_C_fe',
 'O_fe',
 'e_O_fe',
 'nr_O_fe',
 'flag_O_fe',
 'Na_fe',
 'e_Na_fe',
 'nr_Na_fe',
 'flag_Na_fe',
 'Mg_fe',
 'e_Mg_fe',
 'nr_Mg_fe',
 'flag_Mg_fe',
 'Al_fe',
 'e_Al_fe',
 'nr_Al_fe',
 'flag_Al_fe',
 'Si_fe',
 'e_Si_fe',
 'nr_Si_fe',
 'flag_Si_fe',
 'K_fe',
 'e_K_fe',
 'nr_K_fe',
 'flag_K_fe',
 'Ca_fe',
 'e_Ca_fe',
 'nr_Ca_fe',
 'flag_Ca_fe',
 'Sc_fe',
 'e_Sc_fe',
 'nr_Sc_fe',
 'flag_Sc_fe',
 'Ti_fe',
 'e_Ti_fe',
 'nr_Ti_fe',
 'flag_Ti_fe',
 'Ti2_fe',
 'e_Ti2_fe',
 'nr_Ti2_fe',
 'flag_Ti2_fe',
 'V_fe',
 'e_

In [7]:

df_GALAH['SpT2'] = df_GALAH.apply(lambda row: starcolorindexSpT.subclass[round(row.bp_rp, 2)], axis=1)
df_GALAH['Parsec'] =  abs(((1 / df_GALAH['parallax']) * 1000))
df_GALAH['kpc'] =  abs(((1 / df_GALAH['parallax'])))
df_GALAH['LY'] = abs(3261.56/df_GALAH['parallax'])

# calculate absolute magnitude and add a new column (M)
df_GALAH['M'] = df_GALAH['phot_g_mean_mag'] + 5*np.log10(df_GALAH['parallax']) - 10
#df_GALAH['M'].describe()

# add a new column ('L_sun') for luminosity in terms of multiples of solar luminosity
df_GALAH['L_sun'] = np.power(10,[(4.77-m)/2.5 for m in df_GALAH['M']])

# add a new column ('T_K') for effective temperature in Kelvin
df_GALAH['T_K'] = [5601/np.power(c+0.4,2/3) for c in df_GALAH['bp_rp']]

df_GALAH['R_sun']=np.around(np.sqrt(df_GALAH['L_sun'])/(df_GALAH['T_K']/5800)**2, decimals=2) # T_☉ = 5800

print("\nDone!")


Done!


In [8]:
df_GALAH.shape[0]

588571

In [9]:
df_GALAH = df_GALAH[(df_GALAH.flag_sp == 0) & (df_GALAH.flag_fe_h == 0)]
                    
df_GALAH.shape[0]

427984

In [10]:
conditions = [
    (df_GALAH['logg'].astype(float) > 4.20),
    ((df_GALAH['logg'].astype(float) <= 4.20) & (df_GALAH['logg'].astype(float) > 3.90)),
    ((df_GALAH['logg'].astype(float) <= 3.90) & (df_GALAH['logg'].astype(float) > 3.60)),
    ((df_GALAH['logg'].astype(float) < 2.55) & (df_GALAH['logg'].astype(float) > 2.35)),
    (df_GALAH['logg'].astype(float) <= 3.60),
    (df_GALAH['logg'].isna())
]

values = ['main_sequence', 'turnoff', 'subgiants', 'red_clump', 'red_giants', 'null']

df_GALAH['Star_Type'] = np.select(conditions, values)

In [11]:
#df_GALAH.columns.sort_values().tolist()

In [12]:
#set the figure size to be wide in the horizontal direction
#to accommodate the image cutouts alongside the spectra 
#fsize=[20,13]
fsize=[20,8]
#mpl.rcParams['axes.linewidth'] = 0.7
FSIZE=18
LWIDTH=0.5
LABELSIZE=10

In [13]:
def Plot_Spectra(df):
    plt.rcParams.update(Ipython_default)
    plt.clf()

    for idx in range(0, df.shape[0]):
        target = str(df['sobject_id'].iloc[idx])
        subclass = str(df['SpT2'].iloc[idx])
        feh = df['fe_h'].iloc[idx]
        logg = df['logg'].iloc[idx]
        teff = str(df['teff'].iloc[idx])
        ra = str(df['ra'].iloc[idx])
        dec = str(df['dec'].iloc[idx])
        star_type = str(df['Star_Type'].iloc[idx])

        #setup the plot using gridspec
        gs = gridspec.GridSpec
        fig = plt.figure(figsize=fsize)

        gs = gridspec.GridSpec(4,2)
        #B
        axB = fig.add_subplot(gs[0,1])
        #V
        axV = fig.add_subplot(gs[1,1])
        #R
        axR = fig.add_subplot(gs[2,1])
        #I
        axI = fig.add_subplot(gs[3,1])

        show_spectrum(target,[axB,axV,axR,axI],"GALAH DR3 Target: [" + str(int(target)) + "]" + 
            "\nStar_Type: [" + str(star_type) + "]    Subclass: [" + str(subclass[:-1]) + "]" + 
            "    Fe/H: [" + str(round(feh,2))   + "]" +
            "    LogG: [" + str(round(logg,2))  + "]" +
            "    Teff: [" + str(teff[:-4])      + "]" +
            "\nRa: " + str(ra) + 
            "    Dec: " + str(dec))

        #plt.savefig("./Images/" + str(subclass[:-1]) + "_" + str(star_type) + "_LogG[" + str(round(logg,2))  + "]" + "_FeH[" + str(round(feh,2)) + "]" + "_Ra[" + str(ra) + "]" + "_Dec[" + str(dec) + "]" + ".png")
        plt.show()

    print("Done!")
    plt.style.use('dark_background');

In [14]:
def Save_Spectra(df):
    plt.rcParams.update(Ipython_default)
    plt.clf()

    for idx in range(0, df.shape[0]):
        target = str(df['sobject_id'].iloc[idx])
        subclass = str(df['SpT2'].iloc[idx])
        feh = df['fe_h'].iloc[idx]
        logg = df['logg'].iloc[idx]
        teff = str(df['teff'].iloc[idx])
        ra = str(df['ra'].iloc[idx])
        dec = str(df['dec'].iloc[idx])
        star_type = str(df['Star_Type'].iloc[idx])

        #setup the plot using gridspec
        gs = gridspec.GridSpec
        fig = plt.figure(figsize=fsize)

        gs = gridspec.GridSpec(4,2)
        #B
        axB = fig.add_subplot(gs[0,1])
        #V
        axV = fig.add_subplot(gs[1,1])
        #R
        axR = fig.add_subplot(gs[2,1])
        #I
        axI = fig.add_subplot(gs[3,1])

        show_spectrum(target,[axB,axV,axR,axI],"GALAH DR3 Target: [" + str(int(target)) + "]" + 
            "\nStar_Type: [" + str(star_type) + "]    Subclass: [" + str(subclass[:-1]) + "]" + 
            "    Fe/H: [" + str(round(feh,2))   + "]" +
            "    LogG: [" + str(round(logg,2))  + "]" +
            "    Teff: [" + str(teff[:-4])      + "]" +
            "\nRa: " + str(ra) + 
            "    Dec: " + str(dec))

        plt.savefig("./Images/" + str(subclass[:-1]) + "_" + str(star_type) + "_FeH[" + str(round(feh,2)) + "]" + "_LogG[" + str(round(logg,2)) + "]" + "_Ra[" + str(ra) + "]" + "_Dec[" + str(dec) + "]" + ".png", dpi=600)
        plt.close()

    print("Done!")
    plt.style.use('dark_background');

In [15]:
f1 = df_GALAH

main_sequence = f1[f1['logg'].astype(float) > 4.20]
turnoff       = f1[(f1['logg'].astype(float) < 4.20) & (f1['logg'].astype(float) > 3.90)]
subgiants     = f1[(f1['logg'].astype(float) < 3.90) & (f1['logg'].astype(float) > 3.60)]
red_clump     = f1[(f1['logg'].astype(float) < 2.55) & (f1['logg'].astype(float) > 2.35)]
red_clump_id  = red_clump.star_id.astype(str)
reds          = f1[f1['logg'].astype(float) < 3.60]
red_giants    = reds[~reds['star_id'].isin(red_clump_id)]

print("\nStars: " + str(f1.shape[0]))
print("main_sequence: " + str(main_sequence.shape[0]))
print("turnoff: " + str(turnoff.shape[0]))
print("subgiants: " + str(subgiants.shape[0]))
print("red_clump: " + str(red_clump.shape[0]))
print("red_giants: " + str(red_giants.shape[0]))

df_class_O = f1[(f1['SpT2'] >= "O0V") & (f1['SpT2'] <= "O9V")]
df_class_B = f1[(f1['SpT2'] >= "B0V") & (f1['SpT2'] <= "B9V")]
df_class_A = f1[(f1['SpT2'] >= "A0V") & (f1['SpT2'] <= "A9V")]
df_class_F = f1[(f1['SpT2'] >= "F0V") & (f1['SpT2'] <= "F9V")]
df_class_G = f1[(f1['SpT2'] >= "G0V") & (f1['SpT2'] <= "G9V")]
df_class_K = f1[(f1['SpT2'] >= "K0V") & (f1['SpT2'] <= "K9V")]
df_class_M = f1[(f1['SpT2'] >= "M0V") & (f1['SpT2'] <= "M9V")]

print("\nClass O: " + str(df_class_O.shape[0]))
print("Class B: " + str(df_class_B.shape[0]))
print("Class A: " + str(df_class_A.shape[0]))
print("Class F: " + str(df_class_F.shape[0]))
print("Class G: " + str(df_class_G.shape[0]))
print("Class K: " + str(df_class_K.shape[0]))
print("Class M: " + str(df_class_M.shape[0]))


Stars: 427984
main_sequence: 131327
turnoff: 109116
subgiants: 40818
red_clump: 41837
red_giants: 104886

Class O: 472
Class B: 33
Class A: 1631
Class F: 78068
Class G: 134962
Class K: 190926
Class M: 3482


In [16]:
from astroquery.vizier import Vizier
from astropy.table import Table

Vizier.ROW_LIMIT = -1

In [17]:
from astroquery.vizier import Vizier

# GALAH survey, chemodynamical analyse with TGAS (Buder+, 2019)
result = Vizier.query_constraints(catalog='J/A+A/624/A19')

In [ ]:
print(result)

In [ ]:
result[0].columns

In [25]:
list_columns = ['SourceId','_2MASS','GALAH','RAICRS','DEICRS','plx','pmRA','pmDE','Teff','logg','__Fe_H_','vmic','vsini','RV','__alpha_Fe_','LiAbund','CAbund','OAbund','NaAbund','MgAbund','AlAbund','SiAbund','KAbund','CaAbund','ScAbund','TiAbund','VAbund','CrAbund','MnAbund','FeAbund','CoAbund','NiAbund','CuAbund','ZnAbund','RbAbund','SrAbund','YAbund','ZrAbund','MoAbund','RuAbund','BaAbund','LaAbund','CeAbund','NdAbund','SmAbund','EuAbund','Agemean','Massmean']


In [26]:
import pandas as pd

list = []

list = result[result.keys()[0]]

df = pd.DataFrame.from_records(list, columns=list_columns)

In [29]:
df_vizier_table = df_GALAH[df_GALAH['sobject_id'].isin(df['GALAH'])]
len(df_vizier_table)

5429

In [31]:
idx_start = 0
idx_end = 25

In [32]:
#f0 = main_sequence
#f0 = turnoff
#f0 = subgiants
#f0 = red_clump
#f0 = red_giants
#f0 = df_class_O
#f0 = df_class_B
#f0 = df_class_A
#f0 = df_class_F
#f0 = df_class_G
#f0 = df_class_K
#f0 = df_class_M 
#f0 = df_GALAH
f0 = df_vizier_table

f1_sorted = f0.sort_values(by =['fe_h'], ascending=True)

df = f1_sorted[idx_start:idx_end]

print(df[['sobject_id', 'Star_Type', 'SpT2', 'logg', 'fe_h', 'field_id']])

#Plot_Spectra(df)
Save_Spectra(df)

             sobject_id      Star_Type   SpT2      logg      fe_h  field_id
60722   140809003701167  main_sequence    F7V  4.401020 -2.062589       139
2877    131216003201003        turnoff    O3V  4.124098 -2.047103        -1
96940   150206003601197        turnoff    G3V  3.922461 -1.972308      3446
248556  160426003501078      subgiants  F9.5V  3.847551 -1.913672      3548
78943   141104004301029  main_sequence    G0V  4.299097 -1.905546      1012
270715  160524006601089  main_sequence    F9V  4.353731 -1.500562      2922
57999   140808003201002        turnoff    F8V  4.039115 -1.329680        14
118225  150411003601047      subgiants    G7V  3.687579 -1.284641      6635
157059  150827004001280        turnoff    F9V  4.059362 -1.251299        64
105399  150211002201314  main_sequence    F8V  4.278882 -1.243416      2547
74751   141101003001335  main_sequence    F8V  4.360895 -1.239847       211
171658  151009001601086        turnoff    G5V  3.900675 -1.167198      6661
194318  1601

<Figure size 640x480 with 0 Axes>